In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import pandas as pd
import json
import matplotlib.pyplot as plt

# Define the neural network architecture (same as the one used for training)
class ScoringModel(nn.Module):
    def __init__(self, num_inputs=24):  # Adjusted number of inputs
        super(ScoringModel, self).__init__()
        self.layer1 = nn.Linear(num_inputs, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, 64)
        self.layer5 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)  # Binary classification
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.leaky_relu(self.layer1(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer2(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer3(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer4(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer5(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x

# Load the model
model_path = "/Users/marcel/Desktop/project/model/soccer_ai_model1.pt"
model = torch.jit.load(model_path)
model.eval()

# Prepare test dataset
# Assuming you have a test dataset prepared similar to the training dataset
file_path = '/Users/marcel/Desktop/project/model/goal_ticks.csv'  # Update with your test dataset path

# Define column names based on the provided structure
column_names = [
    "kart_ID", "ball_X", "ball_Z", "ball_aim_X", "ball_aim_Z", "ball_node", "previous_X", "previous_Z", "ball_heading",
    "ball_appr_goal", "dist_to_ball", "kart_X", "kart_Z", "vel_X", "vel_Z", "speed", "steer", "accel", "kart_node", 
    "target_encoded", "target_pos_X", "target_pos_Z", "kart0_sector", "kart1_sector", "kart0_X", "kart0_Z", "kart1_X", 
    "kart1_Z", "has_powerup", "goal"
]

# Load the test dataset
df = pd.read_csv(file_path, header=None, names=column_names)

# Drop the specified columns
df = df.drop(columns=["target_encoded", "target_pos_X", "target_pos_Z", "steer", "accel"])

# Extract features and labels
X_test = df.iloc[:, :-1].values  # All columns except the last one (goal)
y_test = df['goal'].values  # The 'goal' column as binary

# Normalize features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Convert to PyTorch tensors
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Prepare DataLoader for test set
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define loss function
criterion = nn.BCELoss()

# Training function with history tracking
def train_model(train_loader, model, criterion, optimizer, num_epochs=10):
    model.train()
    train_loss_history = []
    train_accuracy_history = []

    for epoch in range(num_epochs):
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Compute accuracy
            predicted = (outputs >= 0.5).float()
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        avg_loss = total_loss / len(train_loader)
        accuracy = correct_predictions / total_samples

        train_loss_history.append(avg_loss)
        train_accuracy_history.append(accuracy)

        if (epoch + 1) % 10 == 0:  # Print every 10 epochs
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

    return train_loss_history, train_accuracy_history

# Validation function
def validate_model(val_loader, model, criterion):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Compute accuracy
            predicted = (outputs >= 0.5).float()
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    avg_loss = total_loss / len(val_loader)
    accuracy = correct_predictions / total_samples
    return avg_loss, accuracy

# Plotting function
def plot_metrics(train_loss_history, train_accuracy_history):
    epochs = range(1, len(train_loss_history) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss_history, 'b', label='Training loss')
    plt.title('Training loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracy_history, 'b', label='Training accuracy')
    plt.title('Training accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Load your training data (assuming you have it stored similarly to test data)
# Example:
# df_train = pd.read_csv('path_to_your_train_data.csv', header=None, names=column_names)
# X_train = df_train.iloc[:, :-1].values
# y_train = df_train['goal'].values
# X_train_scaled = scaler.fit_transform(X_train)
# X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Example: Train the model and plot metrics (if you have the training data)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# train_loss_history, train_accuracy_history = train_model(train_loader, model, criterion, optimizer, num_epochs=100)
# plot_metrics(train_loss_history, train_accuracy_history)

# Evaluate the model
avg_loss, accuracy = validate_model(test_loader, model, criterion)
print(f'Average Loss: {avg_loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__.py", line 24, in forward
    leaky_relu = self.leaky_relu
    layer1 = self.layer1
    _0 = (leaky_relu).forward((layer1).forward(x, ), )
                               ~~~~~~~~~~~~~~~ <--- HERE
    _1 = (layer2).forward((dropout).forward(_0, ), )
    _2 = (dropout).forward1((leaky_relu).forward1(_1, ), )
  File "code/__torch__/torch/nn/modules/linear.py", line 12, in forward
    bias = self.bias
    weight = self.weight
    return torch.linear(x, weight, bias)
           ~~~~~~~~~~~~ <--- HERE

Traceback of TorchScript, original code (most recent call last):
/Users/marcel/anaconda3/lib/python3.11/site-packages/torch/nn/modules/linear.py(116): forward
/Users/marcel/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py(1501): _slow_forward
/Users/marcel/anaconda3/lib/python3.11/site-packages/torch/nn/modules/mo